In [3]:
from sudoku import Sudoku
import pandas as pd
import subprocess
import tempfile

In [4]:
#get main cnf file
path = 'sudoku9.cnf'
cnf =""
f = open(path, 'r')
for line in f:
    cnf+= line
f.close()

In [5]:
# Global Constants
MIN_VARS = 1
MAX_VARS = 2
INSTANCES_PER_VAR = 1
SUDOKU_DEGREE = 9
VAR_COUNT = int(str(SUDOKU_DEGREE) + str(SUDOKU_DEGREE) + str(SUDOKU_DEGREE))

In [6]:
frames = [] # will store dataframes for each variable

In [7]:
frames = []
for var in range(MIN_VARS, MAX_VARS):
    boards = []
    frame = pd.DataFrame(columns=['board', 'solution'])
    for y in range(INSTANCES_PER_VAR):
        board = Sudoku(SUDOKU_DEGREE)
        board.randomize_board(var)
        boards.append(board)
    cnfs = [board.cnf_output() for board in boards]
    frame['board'] = boards
    frame['cnf'] = cnfs
    frame['num_vars'] = var
    frames.append(frame)
frames = pd.DataFrame(pd.concat(frames))
frames.head(5)

,board,solution,cnf,num_vars
0,<sudoku.Sudoku object at 0x7f837bc86080>,NaN,607 0\n,1
1,<sudoku.Sudoku object at 0x7f834813cba8>,NaN,666 0\n,1
2,<sudoku.Sudoku object at 0x7f834813cc18>,NaN,275 0\n,1
3,<sudoku.Sudoku object at 0x7f834813cc50>,NaN,506 0\n,1
4,<sudoku.Sudoku object at 0x7f834813cc88>,NaN,062 0\n,1


In [15]:
def get_solution_from_cnf(cnf_string):
    
    cnf_string = cnf + cnf_string #append the whole cnf
    
    bashCommand = 'minisat %s %s' # command to run on the terminal
    
     # Our temporary input/output files 
    input_file = tempfile.NamedTemporaryFile(mode="w")
    output_file = tempfile.NamedTemporaryFile()
    
    # Count number of clauses
    clause_count = len(cnf_string.splitlines())
    
    # writes the file with DIMACS suitable header
    input_file.write("p cnf " + str(VAR_COUNT) + " " + str(clause_count) + "\n" + cnf_string)
 
    # call the bash terminal
    subprocess.call(bashCommand % (input_file.name, output_file.name), shell=True, stdout = subprocess.PIPE)
    
    input_file.close()
    
    return output_file.read().decode("utf-8") 

In [16]:
%%time
frames['solution'] = frames['cnf'].apply(get_solution_from_cnf)

CPU times: user 2.26 s, sys: 4.5 s, total: 6.76 s
Wall time: 8.74 s


In [20]:
frames.head(5)

,board,solution,cnf,num_vars
0,<sudoku.Sudoku object at 0x7f837bc86080>,,607 0\n,1
1,<sudoku.Sudoku object at 0x7f834813cba8>,,666 0\n,1
2,<sudoku.Sudoku object at 0x7f834813cc18>,,275 0\n,1
3,<sudoku.Sudoku object at 0x7f834813cc50>,,506 0\n,1
4,<sudoku.Sudoku object at 0x7f834813cc88>,,062 0\n,1
